# NER

In [2]:
from datasets import load_dataset

lst20 = load_dataset("lst20", data_dir='LST20_Corpus', trust_remote_code=True)
lst20

Generating train split:   0%|          | 0/63310 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5620 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 63310
    })
    validation: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5620
    })
    test: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5250
    })
})

In [3]:
label_list = lst20["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B_BRN',
 'B_DES',
 'B_DTM',
 'B_LOC',
 'B_MEA',
 'B_NUM',
 'B_ORG',
 'B_PER',
 'B_TRM',
 'B_TTL',
 'I_BRN',
 'I_DES',
 'I_DTM',
 'I_LOC',
 'I_MEA',
 'I_NUM',
 'I_ORG',
 'I_PER',
 'I_TRM',
 'I_TTL',
 'E_BRN',
 'E_DES',
 'E_DTM',
 'E_LOC',
 'E_MEA',
 'E_NUM',
 'E_ORG',
 'E_PER',
 'E_TRM',
 'E_TTL']

In [12]:
from transformers import pipeline, AutoTokenizer, AutoModel
from thai2transformers.preprocess import process_transformers

tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', revision='main', model_max_length=416)

In [13]:
example = lst20["train"][0]
tokens = [i.replace('_', '<_>') for i in example['tokens']]
tokenized_input = tokenizer(tokens, is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['<s>',
 '▁',
 'สุรยุทธ์',
 '▁',
 'ยัน',
 '▁',
 'ปฏิเสธ',
 '▁',
 'ลงนาม',
 '<_>',
 '▁',
 '<unk>',
 '<_>',
 '▁',
 'กับ',
 '▁',
 'อียู',
 '▁ไม่',
 '▁',
 'กระทบ',
 '▁',
 'สัมพันธ์',
 '</s>']

In [24]:
def tokenize_and_align_labels(examples):
    tokens  = [[i.replace('_', '<_>') for i in example] for example in examples['tokens']]
    tokenized_inputs = tokenizer(tokens, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_lst20 = lst20.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/63310 [00:00<?, ? examples/s]

Map:   0%|          | 0/5620 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

In [30]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [31]:
import evaluate

seqeval = evaluate.load("seqeval")

In [32]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [33]:
def getmapping(label_list):
    id2label = {}
    label2id = {}
    for i, label in enumerate(label_list):
        id2label[i] = label
        label2id[label] = i
    return id2label, label2id
    
id2label, label2id = getmapping(label_list)
id2label

{0: 'O',
 1: 'B_BRN',
 2: 'B_DES',
 3: 'B_DTM',
 4: 'B_LOC',
 5: 'B_MEA',
 6: 'B_NUM',
 7: 'B_ORG',
 8: 'B_PER',
 9: 'B_TRM',
 10: 'B_TTL',
 11: 'I_BRN',
 12: 'I_DES',
 13: 'I_DTM',
 14: 'I_LOC',
 15: 'I_MEA',
 16: 'I_NUM',
 17: 'I_ORG',
 18: 'I_PER',
 19: 'I_TRM',
 20: 'I_TTL',
 21: 'E_BRN',
 22: 'E_DES',
 23: 'E_DTM',
 24: 'E_LOC',
 25: 'E_MEA',
 26: 'E_NUM',
 27: 'E_ORG',
 28: 'E_PER',
 29: 'E_TRM',
 30: 'E_TTL'}

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

dataset_name = "lst20"
model = AutoModelForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', 
                                                        revision = f'finetuned@{dataset_name}-ner', 
                                                        num_labels=31, 
                                                        id2label=id2label, 
                                                        label2id=label2id)

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/419M [00:00<?, ?B/s]

In [42]:
training_args = TrainingArguments(
    output_dir="my_awesome_lst20_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # save_steps=500,
    # eval_steps=5,
    load_best_model_at_end=True,
    save_total_limit=3,
    resume_from_checkpoint=True,
    logging_steps=2,
    warmup_steps=593
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_lst20["train"],
    eval_dataset=tokenized_lst20["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train(resume_from_checkpoint=True)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
3,0.063700,0.120955,0.756143,0.811519,0.782853,0.958447
4,0.058800,0.122378,0.760603,0.808141,0.783652,0.958896
5,0.057000,0.125901,0.757622,0.806997,0.781530,0.958770


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: E_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.p

TrainOutput(global_step=9895, training_loss=0.03621461508309016, metrics={'train_runtime': 2156.8947, 'train_samples_per_second': 146.762, 'train_steps_per_second': 4.588, 'total_flos': 3.5414584058598732e+16, 'train_loss': 0.03621461508309016, 'epoch': 5.0})

In [43]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: E_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.p

{'eval_loss': 0.11813875287771225,
 'eval_precision': 0.7641824821272437,
 'eval_recall': 0.809612031386225,
 'eval_f1': 0.7862415663447546,
 'eval_accuracy': 0.9595181350649852,
 'eval_runtime': 20.1456,
 'eval_samples_per_second': 260.603,
 'eval_steps_per_second': 4.12,
 'epoch': 5.0}

In [47]:
trainer.model.push_to_hub("ner-wangchanberta-lst20", private=True)
trainer.tokenizer.push_to_hub('ner-wangchanberta-lst20', private=True)

model.safetensors:   0%|          | 0.00/419M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kkatiz/ner-wangchanberta-lst20/commit/403c654ac8817327881876d314be244823e46b8e', commit_message='Upload tokenizer', commit_description='', oid='403c654ac8817327881876d314be244823e46b8e', pr_url=None, pr_revision=None, pr_num=None)